### Experiment with mfcc + noice

In [2]:
import joblib
import librosa
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

In [3]:
csv_path = r"CSVs"
train_df = pd.read_csv(f"{csv_path}/train.csv")
test_df = pd.read_csv(f"{csv_path}/test.csv")
target_length = 41760

In [4]:
def load_audio(file_path):
	audio, sr = librosa.load(file_path, sr=16_000)  # Load audio file
	return audio, sr

# Load all audio files and labels into a list
audio_files = [...]  # List of file paths
labels = [label for label in train_df['word']]  # List of corresponding labels
audio_data = [(load_audio(file)) for file in train_df['path']]

In [5]:
def audio_array(path: str):
	audio, sr = librosa.load(path, sr=16_000)
	return audio, sr

In [6]:
def extract_mfcc(audio, sr, num_mfcc=13, n_fft=2048, hop_length=512, pad_to=None):
	mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
	if pad_to:
		pad_width = pad_to - mfcc.shape[1]
		mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
	return mfcc.flatten()  # Flatten MFCC matrix

# Extract MFCC features for all audio data
mfcc_features = [extract_mfcc(audio, sr, pad_to=target_length) for (audio, sr) in audio_data]

In [7]:
df = pd.DataFrame(mfcc_features)
df['label'] = labels
X = df.drop('label', axis=1)
y = df['label']
y_encoded = y.replace({'Fuck': 0, 'Shit': 1, 'Other': 2})

In [8]:
test_audio_files = [...]  # List of file paths
test_labels = [label for label in test_df['word']]  # List of corresponding labels
test_audio_data = [(load_audio(file)) for file in test_df['path']]

test_mfcc_features = [extract_mfcc(audio, sr, pad_to=target_length) for (audio, sr) in test_audio_data]

test_df = pd.DataFrame(test_mfcc_features)
test_df['label'] = test_labels

In [9]:
test_X = test_df.drop('label', axis=1)
test_y = test_df['label']
test_y_encoded = test_y.replace({'Fuck': 0, 'Shit': 1, 'Other': 2})

In [10]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [30]:
svm_model = SVC(probability=True, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
xgb_model = xgb.XGBClassifier(objective="multi:softmax", num_class=3, random_state=42)
knn_model = KNeighborsClassifier(n_neighbors=3)
lgr_model = LogisticRegression(random_state=42, max_iter=240)

In [20]:
svm_model.fit(X, y)

SVC(probability=True, random_state=42)

In [21]:
rf_model.fit(X, y)

RandomForestClassifier(random_state=42)

In [22]:
xgb_model.fit(X, y_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=3,
              num_parallel_tree=None, ...)

In [23]:
knn_model.fit(X, y)

KNeighborsClassifier(n_neighbors=3)

In [16]:
lgr_model = LogisticRegression(random_state=42, max_iter=500)
lgr_model.fit(X, y)

LogisticRegression(max_iter=500, random_state=42)

In [17]:
joblib.dump(svm_model, r"models\svm_model.pkl")
joblib.dump(rf_model, r"models\rf_model.pkl")
joblib.dump(xgb_model, r"models\xgb_model.pkl")
joblib.dump(knn_model, r"models\knn_model.pkl")
joblib.dump(lgr_model, r"models\lgr_model.pkl")

['models\\lgr_model.pkl']

In [18]:
svm_model = joblib.load(r"models\svm_model.pkl")
rf_model = joblib.load(r"models\rf_model.pkl")
xgb_model = joblib.load(r"models\xgb_model.pkl")
knn_model = joblib.load(r"models\knn_model.pkl")
knn_model = joblib.load(r"models\lgr_model.pkl")

In [19]:
models_list = (svm_model, rf_model, xgb_model, knn_model, lgr_model)
model_names_list = ("svm_model", "rf_model", "xgb_model", "knn_model", "lgr_model")

In [20]:
def evaluate_model(model, model_name, x, y):
	print(f"========{model_name}========")
	# Make predictions
	y_pred = model.predict(x)
	# Evaluate the model
	print(classification_report(y, y_pred))

In [21]:
for i in range(5):
	if model_names_list[i] == "xgb_model":
		y = test_y_encoded
	else:
		y = test_y
	evaluate_model(models_list[i], model_names_list[i], test_X, y)
	print()

========svm_model========
              precision    recall  f1-score   support

        Fuck       0.67      0.50      0.57        20
       Other       0.90      0.90      0.90        20
        Shit       0.64      0.80      0.71        20

    accuracy                           0.73        60
   macro avg       0.74      0.73      0.73        60
weighted avg       0.74      0.73      0.73        60


========rf_model========
              precision    recall  f1-score   support

        Fuck       0.81      0.65      0.72        20
       Other       0.74      1.00      0.85        20
        Shit       0.65      0.55      0.59        20

    accuracy                           0.73        60
   macro avg       0.73      0.73      0.72        60
weighted avg       0.73      0.73      0.72        60


========xgb_model========
              precision    recall  f1-score   support

           0       0.89      0.80      0.84        20
           1       0.84      0.80      0.82       